In [1]:
%load_ext autoreload
%autoreload 2

In [13]:
import datasets
from data.config import config_dict
config = config_dict['wikisimple']
print(config)
data = datasets.load_dataset(
                    # path = str(config.default_cache_dir), 
                    config.dataset_name,
                    data_dir = config.default_cache_dir,
                    num_proc=10,
                    trust_remote_code=True,
                    cache_dir=config.default_cache_dir,
                    download_mode ='reuse_cache_if_exists',
                    **getattr(config, 'kwargs',{}),
                    )


<class 'data.config.WikipediaSimpleConfig'>


Setting num_proc from 10 back to 1 for the train split to disable multiprocessing as it only contains one shard.
Generating train split: 100%|██████████| 205328/205328 [00:01<00:00, 106988.87 examples/s]


: 

In [2]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM
import torch
from model.gpt import GPT
from data.unlabeled import download_dataset
from data.loader import memmapDL, distributedMemmapDL

from model.gpt import GPT

from learner.callbacks import save_checkpoints
from learner.LLMLearner import LLMLearner

from fastai.text.all import *
from fastai.distributed import *
from fastai.callback.wandb import *
import wandb

#________________________________________wandb____________________________________________
log_wandb = False #set to False if you dont want to log progress to W&B

project = 'tinylm' #for wandb
dataset = "wikisimple"
mode = 'scratch'
# ________________________________________hyperparams and settings_________________________

bs=1 #each GPU gets bs = 20, works good for a 24GB GPU
model_id = 'microsoft/Phi-3-mini-4k-instruct'
valid_sampler_size = 1000 #how many samples to use for validation. This is only used to check if validation loss is better than best_valid_loss, so that a checkpoint can be saved. Karpathy uses 200 random points
validate_every = 1000 #1000 iterations, each iteration is bs*total_GPUs inputs
block_size = 1024
qlora = True


/home/agrawalp2/miniconda3/envs/tinylm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# model = GPT.from_hf('microsoft/Phi-3-mini-4k-instruct', enable_qlora=True) 
model = GPT()

In [4]:
model.device

device(type='cpu')

In [5]:

train_path, valid_path = rank0_first(lambda: download_dataset(dataset = dataset, encoder = model.tokenizer)) #check if data exists, download only for rank0 GPU. 


Downloading: 100%|██████████| 1.66k/1.66k [00:00<00:00, 3.66MB/s]
Downloading: 100%|██████████| 235M/235M [00:15<00:00, 15.3MB/s] 


In [6]:
train_dl = memmapDL(train_path, bs = bs, block_size=model.block_size, 
                      dtype=model.tokenizer._get_numpy_dtype())
valid_dl = memmapDL(valid_path, bs = bs, block_size=model.block_size, 
                      dtype=model.tokenizer._get_numpy_dtype(), 
                      sample_size = valid_sampler_size)


In [7]:
dls = DataLoaders(train_dl, valid_dl)

In [8]:
model = GPT()
l=LLMLearner(dls,model,opt_func = partial(OptimWrapper, opt=torch.optim.AdamW),
                loss_func=CrossEntropyLossFlat(), 
                metrics=[accuracy, Perplexity()],)
l.model.device

device(type='cpu')

In [9]:
l.fit(1, find_largest_batch_size = True)

trying 64
trying 32
trying 16
trying 8
Detected largest batch_size = 8


epoch,train_loss,valid_loss,accuracy,perplexity,time


: 